In [1]:
%load_ext autoreload
%autoreload 2

import newdatagen

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-06 18:35:50.266022: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-06 18:35:50.320236: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Repo card metadata block was not found. Setting CardData to empty.


In [2]:
from model.model import load_aligned_model, load_tokenizer

tokenizer = load_tokenizer()
model = load_aligned_model()

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.58s/it]


In [3]:
ds = 'gsm8k'
p = newdatagen.format_entry(newdatagen.dataset_sources[ds][7], ds)
p

'Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?'

In [4]:
from augmentation import generate_cot_completion

generate_cot_completion(p, [], model, tokenizer, debug=3)

[DEBUG3] Input to model:
IMPORTANT: Answer each question properly. Express your answer as either: a single number with no units, commas, or currency symbols; a single capital letter; or a single boolean with the first letter capitalized.

Q: If Alice has 3 apples and Bob gives her 2 more, how many apples does she have?
Reasoning:
Alice starts with 3 apples.
Bob gives Alice 2 additional apples.
Adding 3 and 2 gives the answer.
Answer: 5

Q: If a rectangle has length 8 and width 5, what is its area?
(A) 30   (B) 35   (C) 40   (D) 45
Reasoning:
The formula for area of a rectangle is length × width.
The length is 8 and the width is 5.
Multiplying 8 and 5 gives the answer.
Answer: C

Q: A train leaves at 3 PM and arrives at 6 PM. How many hours long is the trip?
Reasoning:
The train departs at 3 PM.
The train arrives at 6 PM.
The time difference between 3 PM and 6 PM is the answer.
Answer: 3

Q: A factory produces 256 widgets per day. How many widgets will it produce in 365 days?
Reasoning:

(['Ken starts with a final weight of 2 pounds of jelly beans.',
  'Tripling this weight gives another 6 pounds.',
  'At this point, the total weight would be 2 (jelly beans) + 6 (3 of 2 pounds) = 8 pounds.',
  'Next, adding 2 pounds of jelly beans gives an additional 2 pounds. This brings the weight to 10 pounds.',
  'Finally, doubling the weight because of gummy worms gives another 10 (10 original pounds) × 2 = 20 pounds.',
  'The overall weight is thus: 10 (initial weight) + 2 (2nd jelly beans) + 20 (gummy worms).',
  'This adds to 32 pounds.'],
 '32')

In [5]:
RAW_FORMAT_INSTRUCTIONS = """IMPORTANT: Answer each question properly. Express your answer as either: a single number with no units, commas, or currency symbols; a single capital letter; or a single boolean with the first letter capitalized.

Q: If Alice has 3 apples and Bob gives her 2 more, how many apples does she have?
Answer: 5

Q: If a rectangle has length 8 and width 5, what is its area?
(A) 30   (B) 35   (C) 40   (D) 45
Answer: C

Q: A train leaves at 3 PM and arrives at 6 PM. How many hours long is the trip?
Answer: 3

Q: The Earth orbits the Sun once every year. True or False?
Answer: True

Q: %s
Answer:"""

import util
util.prompt_model_answer([RAW_FORMAT_INSTRUCTIONS % p], model, tokenizer, max_new_tokens=50)[0][0].split("\n\n")[0].strip()

'10'